In [398]:
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)

from llama_index.legacy.llms.vertex import Vertex
from langchain.chat_models import ChatVertexAI
from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy import Document
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser


from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import chromadb


from google.oauth2 import service_account
from llama_index.legacy import set_global_service_context
from google.cloud import aiplatform
import json 

## imports from og llama index file

In [399]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy.text_splitter import TokenTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.llms.vertex import Vertex
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)
from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

from llama_index.legacy.postprocessor import NERPIINodePostprocessor, SentenceEmbeddingOptimizer
from llama_index.legacy import ServiceContext
from llama_index.legacy.schema import QueryBundle
from llama_index.legacy.schema import NodeWithScore, TextNode

from pathlib import Path
from tqdm.notebook import tqdm
from google.oauth2 import service_account

from llama_index.legacy import set_global_service_context
import re
import uuid
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
import csv
from typing import List, Tuple, Dict
import time
import json

from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.legacy import StorageContext
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import ChatVertexAI
from llama_index.legacy import Response
from llama_index.legacy.response_synthesizers import Refine
from llama_index.legacy.evaluation import SemanticSimilarityEvaluator
from llama_index.legacy.evaluation import RelevancyEvaluator
from llama_index.legacy.embeddings import SimilarityMode

import nest_asyncio
nest_asyncio.apply()

# additional
from llama_index.legacy.postprocessor import LongContextReorder
from llama_index.legacy.retrievers import AutoMergingRetriever
from llama_index.legacy.query_engine import TransformQueryEngine
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.indices.query.query_transform.base import (
    HyDEQueryTransform,StepDecomposeQueryTransform
)
from llama_index.legacy.node_parser import get_leaf_nodes
from llama_index.legacy import Document
# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
# callback_manager = CallbackManager([llama_debug])

## Initialize credentials, models, and embedding models

In [400]:
import vertexai
from vertexai.language_models import TextGenerationModel
from langchain_google_vertexai.llms import VertexAI

credentials = service_account.Credentials.from_service_account_file("credentials/vertex-test-417403-ce72ad032af7.json")
vertex_ai = Vertex(model="text-bison", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
text_model = TextGenerationModel.from_pretrained("text-bison@002")
text_vertex_ai = VertexAI(model_name="text-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)                       
chat_vertex_ai = ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
langchain_chat_vertex_ai = LangChainLLM(ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2))

selected_model = langchain_chat_vertex_ai

embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest'))

In [401]:
service_context = ServiceContext.from_defaults(llm=vertex_ai, embed_model=embed_model, chunk_size=1024, chunk_overlap=20)
set_global_service_context(service_context)
# need to set global service context becuase if not index = VectorStoreIndex.from_vector_store(vector_store, embed_model=Settings.embed_model) will produce an error
# it will say that it requires OPENAI key etc
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='textembedding-gecko-multilingual@latest', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>, id_func=<function default_id_func at 0x000002D8D4E38860>, chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x000002D8EEA36D90>, callback_manager=<llama_index.legacy.callbacks.base.

In [402]:
import log_appointment as logapp # pingpings file

## {DONE} Produce csv storing query, model response (and soon others)

load query text files into one csv file

In [403]:
# import pandas as pd

# # Sample text strings (replace with your actual file paths)
# text_file1 = "../data/chief complaint.txt"
# text_file2 = "../data/making appointment.txt"
# text_file3 = "../data/medical experts.txt"
# text_file4 = "../data/manual questions.txt"

# # Create a list of text strings
# text_list = []
# for filename in [text_file1, text_file2, text_file3, text_file4]:
#   with open(filename, "r") as f:
#     # Read all lines and concatenate them into a single string
#     text_content = f.readlines()  # Remove potential leading/trailing whitespace
#     text_list.extend(text_content)

# text_list = [line.replace('\n', '') for line in text_list]
# print(len(text_list))
# # text_list

In [404]:
# Create a DataFrame with a single column named "Text"
# df = pd.DataFrame({"Text": text_list}, columns=["query"])
# df = pd.DataFrame(text_list, columns=["query"])

# filepath = "model_response.csv"
# df.to_csv(filepath, index=False)
# print(f"Data saved to {filepath}")

Create dataframe from model_response.csv file

In [405]:
filepath = "model_response.csv"
model_response_df = pd.read_csv(filepath)
model_response_df

,query
0,ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควร...
1,ผมมีอาการท้องผูกและอาหารย่อยยากมานานหลายปีต้อง...
2,ผู้สูงวัยมีอาการปวดศีรษะรุนแรงและมีอาการคลื่นไ...
3,หมอแบบไหนที่ให้คำปรึกษาได้หากมีอาการท้องเสียรุ...
4,คลินิกด้านไหนที่รักษาอาการระบบทางเดินหายใจอักเ...
5,อยากตรวจหาสาเหตุของการเจ็บแสบหลังสะโพกและมีปัญ...
6,เด็กมีปัญหาการมองเห็นและมีปัญหาด้านสายตาต้องปร...
7,น้องสาวมีอาการตกขาวที่มีกลิ่นเหม็นและคันแต่อาย...
8,ลูกสาวมีอาการปวดท้องประจำเดือนและการมีรอยคราบเ...
9,อยากหาหมอที่ตรวจหาสาเหตุของอาการหัวใจเต้นเร็วแ...


Create qna dataframe from medical_experts_ans.csv (for appointments)

In [406]:
# filepath = "medical_experts_ans.csv"
# model_response_df = pd.read_csv(filepath)
# model_response_df

Create log_appointment.py log file

In [407]:
log_path = "../data/log_appointment.csv"
logapp.create_appoint_log(num_week=5, log_path=log_path, delete_existing=True)

## Import and load data
- How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

### Load data from doctor_db.csv (modified from siriraj_doctor_details.csv for appointments) 

In [408]:
import pandas as pd

# unused
# reader = SimpleDirectoryReader(
#     input_files=["../data/siriraj_doctor_details.csv"]
# )
# csv_data = reader.load_data()
# print(f"Loaded {len(csv_data)} docs")
# csv_data

doctor_csv_file_path = "../data/doctor_db.csv"
doctor_csv_df = pd.read_csv(doctor_csv_file_path)
columns_to_drop = ['image_src', 'url', 'qualification']
doctor_csv_df = doctor_csv_df.drop(columns=[col for col in doctor_csv_df.columns if col in columns_to_drop])
doctor_csv_df

,name,table_check,expertise,doctorID
0,รศ. พญ.กติกา นวพันธุ์,"เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",เวชศาสตร์มารดาและทารกในครรภ์,0
1,รศ. พญ.กนกรัตน์ พรพาณิชย์,"พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5",ศัลยกรรมกระดูกเบ้าตาและตกแต่ง,1
2,อ. พญ.กนกรัตน์ สุวรรณสิทธิ์,"พฤหัสบดี 13:30 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",โสต ประสาท การได้ยินและการทรงตัว,2
3,ทพญ.กนกอร ตันติพาณิชย์กูล,"พุธ 16:30 - 19:30 น. สัปดาห์ที่ 3,4",ทันตกรรมจัดฟัน การจัดฟันด้วยการผ่าตัดจัดฟัน,3
4,ผศ. นพ.กมล เผือกเพ็ชร,"พุธ 16:00 - 18:00 น. สัปดาห์ที่ 1,3,5 อาทิตย์ ...",กุมารเวชศาสตร์โลหิตวิทยาและมะเร็งในเด็ก,4
...,...,...,...,...
625,นพ.เอกราช ลิ่วธนมงคล,"เสาร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 13...","โรคของลำไส้ใหญ่และทวารหนัก, ฝีคัณฑสูตร, ริดสีด...",625
626,รศ. นพ.เอกรินทร์ โชติกวาณิชย์,"พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร...","โรคนิ่วไต ท่อไต และกระเพาะปัสสาวะ, โรคเนื้องอก...",626
627,อ. นพ.เอกวิทย์ เกยุราพันธุ์,"ศุกร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 อา...",ศัลยศาสตร์ออร์โธปิดิกส์,627
628,นพ.เอนกชัย ดกพฤกษ์,NaN,เวชศาสตร์ฉุกเฉิน,628


### Load data from clinic-list-final.csv


In [409]:
clinic_csv_file_path = "../data/clinic-list-final.csv"
clinic_csv_df = pd.read_csv(clinic_csv_file_path)
columns_to_drop = ['link']
clinic_csv_df = clinic_csv_df.drop(columns=[col for col in clinic_csv_df.columns if col in columns_to_drop])
clinic_csv_df

,office_hrs,location,phone,clinic_name
0,วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.,ชั้น 4 โซน E,โทร:1474กด 1,คลินิกตรวจสุขภาพ
1,วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมง,ชั้น 1 โซน A,โทร:1474กด 1,คลินิกผู้ป่วยฉุกเฉิน
2,วันจันทร์-วันอาทิตย์เวลา 08:00-16:00 น.,ชั้น 4 โซน C,โทร:1474กด 1,คลินิกฟื้นฟูหัวใจ
3,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 4 โซน D,โทร:1474กด 1,คลินิกเบาหวาน ไทรอยด์ และต่อมไร้ท่อ
4,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ตา
5,วันจันทร์-วันเสาร์เวลา 07:00-21:00 น.วันอาทิตย...,ชั้น 3 โซน A,โทร:1474กด 1,ศูนย์ทันตกรรม
6,วันจันทร์-วันอาทิตย์เวลา 07:00-19:00 น.,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ทางเดินปัสสาวะ
7,วันจันทร์-วันศุกร์ เวลา 07:00-21:00 น.วันเสาร์...,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ทางเดินอาหารและตับ
8,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 2 โซน E,โทร:1474กด 1,ศูนย์นรีเวช
9,วันจันทร์-วันอาทิตย์เวลา 08:00-19:00 น.,ชั้น 3 โซน A,โทร:1474กด 1,ศูนย์ผิวหนังและศัลยกรรมตกแต่ง


### {UNUSED} Load data from siriraj_doctor_details.csv (given)

Cleaning csv data

In [410]:
# import pandas as pd

# # unused
# # reader = SimpleDirectoryReader(
# #     input_files=["../data/siriraj_doctor_details.csv"]
# # )
# # csv_data = reader.load_data()
# # print(f"Loaded {len(csv_data)} docs")
# # csv_data

# csv_file_path = "../data/siriraj_doctor_details.csv"
# csv_df = pd.read_csv(csv_file_path)
# columns_to_drop = ['image_src', 'url', 'qualification']
# df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
# df

### TODO: Cluster expertise (not done) << FULL CODE IS IN MINNIELLM. IT IS REMOVED IN THIS FILE
- not sure if needed atm. will be used as a point of improvement later

In [411]:
clinic_csv_data = clinic_csv_df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
doctor_csv_data = doctor_csv_df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
clinic_csv_data = [Document(text=t) for t in clinic_csv_data] # produce Documents from csv data
doctor_csv_data = [Document(text=t) for t in doctor_csv_data] # produce Documents from csv data
print(len(clinic_csv_data))
print(clinic_csv_data)

22
[Document(id_='fb8a1ebb-a7d4-4c17-a463-78f571f6b978', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.ชั้น 4 โซน Eโทร:1474กด 1คลินิกตรวจสุขภาพ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='0099016f-a2d0-470d-992d-4c5132735a8d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมงชั้น 1 โซน Aโทร:1474กด 1คลินิกผู้ป่วยฉุกเฉิน', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='566f0ffc-7dc3-42ac-ab9b-9975720d0a2f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 08:

In [412]:
print(len(doctor_csv_data))
print(doctor_csv_data)

630
[Document(id_='59d8ae96-fd38-480a-a336-53a1c97fe030', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กติกา นวพันธุ์เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5เวชศาสตร์มารดาและทารกในครรภ์0', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='72aa3561-4eb7-4ff7-a111-50346886d14f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กนกรัตน์ พรพาณิชย์พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5ศัลยกรรมกระดูกเบ้าตาและตกแต่ง1', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='9a957adc-4d94-4b14-bfa6-72df9c6e6922', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, te

### Combine all loaded data into singular Document list docs

In [413]:
docs = []
docs.extend(clinic_csv_data)
# docs.extend(doctor_csv_data)


print(len(docs))
print(docs)

22
[Document(id_='fb8a1ebb-a7d4-4c17-a463-78f571f6b978', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.ชั้น 4 โซน Eโทร:1474กด 1คลินิกตรวจสุขภาพ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='0099016f-a2d0-470d-992d-4c5132735a8d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมงชั้น 1 โซน Aโทร:1474กด 1คลินิกผู้ป่วยฉุกเฉิน', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='566f0ffc-7dc3-42ac-ab9b-9975720d0a2f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 08:

### Create local ChromaDB Vectore Store (currently unused)

In [414]:
# # Create index and persist to Chroma vector store

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents = docs, storage_context=storage_context, embed_model=embed_model
# )

In [415]:
# Load index from persisted vector store

# currently index is used for retriever and not query engine

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# chromadb_index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

# chromadb_query_engine = chromadb_index.as_query_engine(similarity_top_k=2)
# vector_retriever = chromadb_index.as_retriever(similarity_top_k=2)

### Configuring query engine components

Hierarchical Node Parsing

In [416]:
# node_parser = HierarchicalNodeParser.from_defaults(
#     chunk_sizes=[1024, 512, 256]
# )
# nodes = node_parser.get_nodes_from_documents(docs)

# leaf_nodes = get_leaf_nodes(nodes)
# nodes_by_id = {node.node_id: node for node in nodes}

# auto_merging_context = ServiceContext.from_defaults(
#     llm=selected_model,
#     embed_model= embed_model,
#     node_parser=node_parser,
# )

Store configured components in vector store called merging_index

In [417]:
# storage_context = StorageContext.from_defaults()
# storage_context.docstore.add_documents(nodes)

# # automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)
# clinic_doctor_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)

# # TODO: fix the code to store in chromadb
# # currently chromadb is not used

# # merging_index_dir = "./merging_index"
# # siriraj_doctor_index_dir = "./siriraj_doctor_index"
# clinic_doctor_index_dir = "./clinic_doctor_index"
# # clinic_index_dir = "./clinic_index"


# clinic_doctor_index.storage_context.persist(persist_dir=clinic_doctor_index_dir)

Load automerging index

In [418]:
# merging_index_dir = "./merging_index"
# siriraj_doctor_index_dir = "./siriraj_doctor_index"
clinic_doctor_index_dir = "./clinic_doctor_index"
# clinic_index_dir = "./clinic_index"

# loaded_storage_context = StorageContext.from_defaults(persist_dir=merging_index_dir) # load the existing index
loaded_storage_context = StorageContext.from_defaults(persist_dir=clinic_doctor_index_dir) # load the existing index
# automerging_index = load_index_from_storage(loaded_storage_context)
clinic_doctor_index = load_index_from_storage(loaded_storage_context)


In [419]:
# automerging_as_retriever = automerging_index.as_retriever(similarity_top_k=10)
# automerging_retriever = AutoMergingRetriever(
#     automerging_as_retriever, 
#     automerging_index.storage_context, 
#     verbose=True
# )

automerging_as_retriever = clinic_doctor_index.as_retriever(similarity_top_k=10)
automerging_retriever = AutoMergingRetriever(
    automerging_as_retriever, 
    clinic_doctor_index.storage_context, 
    verbose=True
)

### Query engine

In [420]:
reorder = LongContextReorder()
hyde = HyDEQueryTransform(llm=vertex_ai, include_original=True)
retriever_query_engine = RetrieverQueryEngine.from_args(automerging_retriever,
                                              node_postprocessors=[reorder],
                                              )
transform_query_engine = TransformQueryEngine(retriever_query_engine, query_transform=hyde)

In [421]:
transform_query_engine.__dict__

{'_query_engine': <llama_index.legacy.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x2d89da5e350>,
 '_query_transform': <llama_index.legacy.indices.query.query_transform.base.HyDEQueryTransform at 0x2d8813db110>,
 '_transform_metadata': None,
 'callback_manager': <llama_index.legacy.callbacks.base.CallbackManager at 0x2d89da5e4d0>}

In [422]:
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_prompt_str = (
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

TESTING: Response production

In [423]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response=transform_query_engine.query(query)
print("response: ", response.response)
# print("source nodes: ", response.source_nodes[0])
# print("source node text: ", response.source_nodes[0].node.text)
# print("number of source nodes: ", len(response.source_node))

> Merging 1 nodes into parent node.
> Parent node id: 0dc309e4-ad23-4ac6-ae8c-7f34fb35f777.
> Parent node text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43

> Merging 1 nodes into parent node.
> Parent node id: 6616b52b-c9cb-447f-93b6-bd52272f0e46.
> Parent node text: ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:00 - 11:00 น...

> Merging 1 nodes into parent node.
> Parent node id: 8da0c5ff-5ebb-4f43-b00c-54a1105099a4.
> Parent node text: ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ โรคปอดและเวชศาสตร์วิกฤต352

> Merging 1 nodes into parent node.
> Parent node id: 26846bd1-551f-4548-b269-68acc11abb09.
> Parent node text: พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดินหายใจ535

> Merging 1 nodes into parent node.
> Parent node id: d240f0c5-02fd-4711-a8aa-24301de962d3.
> Parent node text: รศ. นพ.ชุณหเกษม   โชตินัยวัตรกุลอังคาร 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 13:00 - 16:00

Inspecting response and context nodes

In [424]:
response.response

' ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์'

In [425]:
print(response.source_nodes[0])

Node ID: 6c5eef22-d51d-4673-ac33-99368cd58c6a
Text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด,
หลอดเลือดหัวใจตีบและลิ้นหัวใจ43
Score:  0.728



In [426]:
response.source_nodes[0].node.text

'รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43'

In [427]:
len(response.source_nodes)

10

Context Transformations - PII Example
- adding context to model response generation

In [428]:
pii_processor = NERPIINodePostprocessor()

def filter_pii_fn(**kwargs):
    # run optimizer
    query_bundle = QueryBundle(query_str=kwargs["query_str"])

    new_nodes = pii_processor.postprocess_nodes(
        [NodeWithScore(node=TextNode(text=kwargs["context_str"]))],
        query_bundle=query_bundle,
    )
    new_node = new_nodes[0]
    return new_node.get_content()

In [429]:
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "(day) (time) (full name of doctor) (doctor's expertise)\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    # "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl_str2 = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "{day_time} {full_name} ({expertise})\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
    "1. {day_time} {full_name} ({expertise})\n"
    "2. {day_time} {full_name} ({expertise})\n"
    "..."
)

In [430]:
import llama_index.legacy.response_synthesizers

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str2, function_mappings={"context_str": filter_pii_fn}
)

prompts_dict = transform_query_engine.get_prompts()
print(list(prompts_dict.keys()))

transform_query_engine.update_prompts(
    {"query_engine:response_synthesizers": qa_prompt_tmpl}
) # use merging index


['query_transform:hyde_prompt', 'query_engine:response_synthesizer:text_qa_template', 'query_engine:response_synthesizer:refine_template']


### Produce dataframe containing query, model response, and context

In [431]:
def produce_response(query):
    # response = chromadb_query_engine.query(query)
    response = transform_query_engine.query(query)
    return response

def produce_context(query):
    # retrieved_nodes = vector_retriever.retrieve(query)
    retrieved_nodes = automerging_retriever.retrieve(query)
    context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
    return context_str

In [432]:
model_response_df

,query
0,ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควร...
1,ผมมีอาการท้องผูกและอาหารย่อยยากมานานหลายปีต้อง...
2,ผู้สูงวัยมีอาการปวดศีรษะรุนแรงและมีอาการคลื่นไ...
3,หมอแบบไหนที่ให้คำปรึกษาได้หากมีอาการท้องเสียรุ...
4,คลินิกด้านไหนที่รักษาอาการระบบทางเดินหายใจอักเ...
5,อยากตรวจหาสาเหตุของการเจ็บแสบหลังสะโพกและมีปัญ...
6,เด็กมีปัญหาการมองเห็นและมีปัญหาด้านสายตาต้องปร...
7,น้องสาวมีอาการตกขาวที่มีกลิ่นเหม็นและคันแต่อาย...
8,ลูกสาวมีอาการปวดท้องประจำเดือนและการมีรอยคราบเ...
9,อยากหาหมอที่ตรวจหาสาเหตุของอาการหัวใจเต้นเร็วแ...


Produce model response for all queries

In [433]:
model_response_df['model_response'] = model_response_df['query'].apply(produce_response)
time.sleep(5)
model_response_df['context'] = model_response_df['query'].apply(produce_context)

> Merging 1 nodes into parent node.
> Parent node id: 0dc309e4-ad23-4ac6-ae8c-7f34fb35f777.
> Parent node text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43

> Merging 1 nodes into parent node.
> Parent node id: 26846bd1-551f-4548-b269-68acc11abb09.
> Parent node text: พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดินหายใจ535

> Merging 1 nodes into parent node.
> Parent node id: 8da0c5ff-5ebb-4f43-b00c-54a1105099a4.
> Parent node text: ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ โรคปอดและเวชศาสตร์วิกฤต352

> Merging 1 nodes into parent node.
> Parent node id: 6616b52b-c9cb-447f-93b6-bd52272f0e46.
> Parent node text: ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:00 - 11:00 น...

> Merging 1 nodes into parent node.
> Parent node id: f8bc1355-5fec-4015-ad2a-79bdec5fe38b.
> Parent node text: รศ. ดร. พญ.เบญจมาศ ช่วยชูnanอายุรกรรมโรคทรวงอกและทางเดินหายใจ254

> Merging 1 nodes into p

> Merging 1 nodes into parent node.
> Parent node id: 4b5f66e7-ba4a-4b92-8642-531946d8bd26.
> Parent node text: อ. พญ.ชวนนท์ พิมลศรีอังคาร 16:00 - 18:30 น. สัปดาห์ที่ 2,3,4,5 16:00 - 18:30 น. สัปดาห์ที่ 1 อาทิ...

> Merging 1 nodes into parent node.
> Parent node id: a50deabb-20a9-4e27-aff7-f77d7eb101a9.
> Parent node text: รศ. นพ.จุลจักร ลิ่มศรีวิไลพฤหัสบดี 16:00 - 19:00 น. สัปดาห์ที่ 1,2,3,4,5 ศุกร์ 09:30 - 12:30 น. ส...

> Merging 1 nodes into parent node.
> Parent node id: 0dc309e4-ad23-4ac6-ae8c-7f34fb35f777.
> Parent node text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43

> Merging 1 nodes into parent node.
> Parent node id: 44ef6ff9-04fd-4ff1-aab5-19c3ebcdc502.
> Parent node text: ผศ. พญ.ปวีณา พิทักษ์สุรชัยจันทร์ 13:00 - 15:00 น. สัปดาห์ที่ 1,2,3,4,5 พุธ 16:00 - 18:00 น. สัปดา...

> Merging 1 nodes into parent node.
> Parent node id: 2eb99111-78f6-445d-83e0-a6a4e570c41e.
> Parent node text: อ. นพ.ปรัชญา มณีประสพโชคศุกร์ 14:00 

In [434]:
model_response_df

,query,model_response,context
0,ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควร...,ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์,พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดิ...
1,ผมมีอาการท้องผูกและอาหารย่อยยากมานานหลายปีต้อง...,คุณอาจมีอาการของโรคลำไส้แปรปรวน (IBS) ซึ่งเป็...,รศ. นพ.จุลจักร ลิ่มศรีวิไลพฤหัสบดี 16:00 - 19:...
2,ผู้สูงวัยมีอาการปวดศีรษะรุนแรงและมีอาการคลื่นไ...,อายุรศาสตร์ผู้สูงอายุ,อ. พญ.ชวนนท์ พิมลศรีอังคาร 16:00 - 18:30 น. สั...
3,หมอแบบไหนที่ให้คำปรึกษาได้หากมีอาการท้องเสียรุ...,จากข้อมูลที่กำหนด ไม่มีข้อมูลเกี่ยวกับอาการท้...,รศ. นพ.จุลจักร ลิ่มศรีวิไลพฤหัสบดี 16:00 - 19:...
4,คลินิกด้านไหนที่รักษาอาการระบบทางเดินหายใจอักเ...,อายุรศาสตร์โรคระบบทางเดินหายใจ (Pulmonary Dis...,พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดิ...
5,อยากตรวจหาสาเหตุของการเจ็บแสบหลังสะโพกและมีปัญ...,จากอาการที่คุณเล่ามา คุณอาจมีปัญหาเกี่ยวกับระ...,อ. นพ.ปราโมทย์ เอื้อโสภณเสาร์ 09:00 - 12:00 น....
6,เด็กมีปัญหาการมองเห็นและมีปัญหาด้านสายตาต้องปร...,จักษุเด็กและตาเหล่,รศ. พญ.ชารีนันท์ จิรภาไพศาลศุกร์ 09:00 - 12:00...
7,น้องสาวมีอาการตกขาวที่มีกลิ่นเหม็นและคันแต่อาย...,จากอาการที่น้องสาวเล่ามา มีความเป็นไปได้ว่าอา...,รศ. นพ.ปารยะ อาศนะเสนเสาร์ 14:30 - 17:00 น. สั...
8,ลูกสาวมีอาการปวดท้องประจำเดือนและการมีรอยคราบเ...,จากอาการปวดท้องประจำเดือนและการมีรอยคราบเลือด...,"สัปดาห์ที่ 1,2,3,4,5ฝีที่ผิวหนัง, แผลติดเชื้อ,..."
9,อยากหาหมอที่ตรวจหาสาเหตุของอาการหัวใจเต้นเร็วแ...,จากข้อมูลที่ให้มา คุณสามารถไปพบแพทย์ได้ดังนี้...,ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:...


record to csv

In [435]:
fp = "mod08_test_clinics_doctors_only_prompten.csv"
model_response_df.to_csv(fp, index=False, mode='w')  # Set index=False to exclude the index column
print("CSV file saved:", fp)

CSV file saved: mod08_test_clinics_doctors_only_prompten.csv


### Testing response production

In [436]:
# query1 = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
# query2 = "ดิฉันต้องการนัดพบนักจิตวิทยาเพื่อการปรึกษาเรื่องความเครียด สะดวกมาพบเฉพาะสัปดาห์แรกและสัปดาห์สุดท้ายของเดือน สามารถมาหาหมอได้วันไหนบ้าง"
# query3 = "อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามากปนกับปัสสาวะ มีปัสสาวะแสบและปวดท้องน้อย"
appointment_q = "ช่วยจัดสรรเวลาเพื่อการปรึกษาเรื่องการเตรียมตัวก่อนการผ่าตัดหัวใจให้หน่อย ผมต้องการทราบวันที่ว่างของแพทย์เพื่อจัดเตรียมตารางเวลาของผมให้เหมาะสม"

response = produce_response(appointment_q)


> Merging 1 nodes into parent node.
> Parent node id: 6616b52b-c9cb-447f-93b6-bd52272f0e46.
> Parent node text: ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:00 - 11:00 น...

> Merging 1 nodes into parent node.
> Parent node id: b74f10b7-a585-46eb-9faf-f57979ae4125.
> Parent node text: ผศ. นพ.ปุณณฤกษ์ ทองเจริญพฤหัสบดี 12:00 - 14:30 น. สัปดาห์ที่ 1,2,3,4,5 12:00 - 14:30 น. สัปดาห์ที...

> Merging 1 nodes into parent node.
> Parent node id: fb1d303f-df89-4a3d-a014-fc0b27046813.
> Parent node text: ศ.คลินิก นพ.ดำรัส ตรีสุโกศลศุกร์ 08:30 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 13:00 - 15:00 น. สัปดาห์ที...

> Merging 1 nodes into parent node.
> Parent node id: 0a9f6ff4-bb1d-48c3-8200-04eca338dda2.
> Parent node text: นพ.ปัญเกียรติ โตพิพัฒน์พุธ 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 ศุกร์ 09:00 - 12:00 น. สัปดาห์ที...

> Merging 1 nodes into parent node.
> Parent node id: c7fd2123-9675-4844-ac24-049033515156.
> Parent node text: นพ.เอก เพ็ชรดาชัยพุธ 11:00 - 11:

In [437]:
print("response:" , response)

response:  - ผศ. นพ.ปุณณฤกษ์ ทองเจริญ: วันพฤหัสบดี เวลา 12:00 - 14:30 น. (สัปดาห์ที่ 1,2,3,4,5)
- ศ.คลินิก นพ.สมชาย ศรียศชาติ: วันพุธ เวลา 10:00 - 11:00 น. (สัปดาห์ที่ 1,2,3,4,5)


In [438]:
context =  str(produce_context(query))
print("context: " , context)

> Merging 1 nodes into parent node.
> Parent node id: 26846bd1-551f-4548-b269-68acc11abb09.
> Parent node text: พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดินหายใจ535

> Merging 1 nodes into parent node.
> Parent node id: f8bc1355-5fec-4015-ad2a-79bdec5fe38b.
> Parent node text: รศ. ดร. พญ.เบญจมาศ ช่วยชูnanอายุรกรรมโรคทรวงอกและทางเดินหายใจ254

> Merging 1 nodes into parent node.
> Parent node id: 8da0c5ff-5ebb-4f43-b00c-54a1105099a4.
> Parent node text: ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ โรคปอดและเวชศาสตร์วิกฤต352

> Merging 1 nodes into parent node.
> Parent node id: fc9a6ef5-7ac3-460f-adcf-f28a2a24938b.
> Parent node text: นพ.ประทีป รักษาสกุลวงศ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ264

> Merging 1 nodes into parent node.
> Parent node id: 229b7540-57d1-4398-8ed5-c25e7459b94c.
> Parent node text: รศ. นพ.บวรฤทธิ์ จักรไพวงศ์อังคาร 07:30 - 08:30 น. สัปดาห์ที่ 1,2,3,4,5การผ่าตัดโรคกระดูกและข้อที่...

> Merging 1 nodes into parent node.
> Parent node id: 9df044f

record to csv

In [439]:
import pandas as pd

data = [query, response, context]
df = pd.DataFrame([data], columns=["query", "response", "context"])

fp = "test_results.csv"
df.to_csv(fp, mode='a', header=False, index=False)
print(f"Data saved to {fp}")

Data saved to test_results.csv
